In [1]:
import numpy as np
from matplotlib import pyplot as plt
from scipy import linalg as la
import pandas as pd

## Exercise 1

In [2]:
def drazintest(A, k, AD):
    '''
    function accepts square matrices A, AD, and k the index of A.
    (Question - is there an algorithm to compute the index 
    of a matrix?)
    Returns Boolean for statement (AD is drazin inverse of A)
    '''
    output = (np.allclose(A @ AD, AD @ A) & \
              np.allclose(np.linalg.matrix_power(A, k+1) @ AD, np.linalg.matrix_power(A, k)) & \
              np.allclose(AD @ A @ AD, AD))
    return output
    
A = np.array([[1, 3, 0, 0], \
              [0, 1, 3, 0], \
              [0, 0, 1, 3], \
              [0, 0, 0, 0]] )
Ak = 1
AD = np.array([[1, -3, 9, 81], \
               [0, 1, -3, -18], \
               [0, 0, 1, 3], \
               [0, 0, 0, 0]])

B = np.array([[1, 1, 3], \
               [5, 2, 6], \
               [-2, -1, -3]])
Bk = 3
BD = np.zeros((3, 3))

drazintest(A, Ak, AD)
drazintest(B, Bk, BD)

True

## Exercise 2

In [12]:
A = np.random.rand(5, 5)
tol = .3

def drazin(A, tol):    
    n = A.shape[0]
    # Sort schur decomposition so that zero eigenvals come last
    f1 = lambda x: abs(x) > tol
    Q1, S, k1 = la.schur(A, sort = f1)
    f2 = lambda x: abs(x) <= tol
    Q2, T, k2 = la.schur(A, sort = f2)
    U = np.hstack((S[:, :k1], T[:,:(n-k1)]))
    #Compute Drazin inverse as in 15.2
    Uinv = np.linalg.inv(U)
    V = Uinv @ A @ U
    Z = np.zeros((n, n), dtype =float)
    if k1 == 0:
        print('ERROR: All zero eigenvalues')
    else:
        Minv = np.linalg.inv(V[:k1,:k1])
        Z[:k1,:k1] = Minv
    out = U @ Z @ Uinv
    return out

#print("\n A = \n", A, "\n T1 = \n", T1, "\n Z1TZ1^-1 = \n", U1 @ T1 @ U1inv)
A = np.array([[1, 3, 0, 0], \
              [0, 1, 3, 0], \
              [0, 0, 1, 3], \
              [0, 0, 0, 0]] )
drazin(A, .01)
# I am not sure why this algorithm is not working...

array([[  1.,  -3.,   9.,  81.],
       [  0.,   1.,  -3., -18.],
       [  0.,   0.,   1.,   3.],
       [  0.,   0.,   0.,   0.]])

## Exercise 3

In [9]:
def drazin2(A, tol):
    n, n = A.shape
    f = lambda x: abs(x) > tol
    g = lambda x: abs(x) <= tol
    Q1, S, k1 = la.schur(A, sort=f)
    Q2, T, k2 = la.schur(A, sort=g)
    U = np.hstack((S[:, :k1], T[:, :(n - k1)]))
    U_inv = la.inv(U)
    V = U_inv @ A @ U
    Z = np.zeros((n, n))
    if k1 != 0:
        M_inv = la.inv(V[:k1, :k1])
        Z[:k1, :k1] = M_inv
#     else:
#         raise ValueError()
    return U @ Z @ U_inv

In [10]:
def getresist(A):
    '''
    Function takes in an adjacency matrix (square matrix, 
    each vertex is the number of connections from one node 
    to another) and returns the effective resistance for the
    graph.
    '''
    n = A.shape[0]
    #Defaine degree matrix
    deg = np.sum(A, axis = 0)
    D = np.diag(deg)
    # Create laplacian
    L = D - A
    I = np.identity(n)
    R = np.zeros((n,n))
    for j in range(n):
        Lj = L.copy()
        Lj[:, j] = I[:, j]
        LjD = drazin(Lj, .01)
        R[:, j] = LjD.diagonal()
        R[j, j] = 0
    return R

#Credit to reiko for typing out all these adjacency matrices


In [6]:
G1 = np.array([[0, 1, 0, 0], [1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0]])
getresist(G1)

NameError: name 'k' is not defined

In [212]:
G2 = np.array([[0, 1], [1, 0]])
getresist(G2)

array([[0., 1.],
       [1., 0.]])

In [213]:
G3 = np.array([[0, 1, 1], [1, 0, 1], [1, 1, 0]])
getresist(G3)


array([[0.        , 0.66666667, 0.66666667],
       [0.66666667, 0.        , 0.66666667],
       [0.66666667, 0.66666667, 0.        ]])

In [214]:
G4 = np.array([[0, 3], [3, 0]])
getresist(G4)

array([[0.        , 0.33333333],
       [0.33333333, 0.        ]])

In [215]:
G5 = np.array([[0, 2], [2, 0]])
getresist(G5)

array([[0. , 0.5],
       [0.5, 0. ]])

In [216]:
G6 = np.array([[0, 4], [4, 0]])
getresist(G6)

array([[0.  , 0.25],
       [0.25, 0.  ]])

## Exercise 4 + 5

In [324]:
class LinkPredictor:
    '''
    Link Predictor class. Accepts filename as an input.
    The format of the file is ""
    '''
    def __init__(self, file):
        #Get raw data
        self.raw = pd.read_csv(file, header=None)
        #Grab names
        self.names = np.unique(np.concatenate((self.raw[0], self.raw[1])))
        n = len(self.names)
        # Create adjacency matrix
        adj = np.zeros((n, n))
        for i in self.raw.index:
            name1, name2 = self.raw[0][i], self.raw[1][i]
            name1loc = np.where(name1 == self.names)
            name2loc = np.where(name2 == self.names)
            adj[name1loc, name2loc] = 1
            adj[name2loc, name1loc] = 1
        self.adj = adj
        #Compute resistance matrix
        self.resist = getresist(self.adj)
        
    def predict_link(self, node='none'):
        '''
        Parameter node = none or the string representative
        of a node. if node = none, return a tuple of the names
        of the nodes between which the next link should occur, 
        if node = name return the name of the next node which
        name should be connected to
        '''
        #First step - zero out all connected nodes
        possconnectmat = self.resist * (1 - self.adj)
        if (node == 'None')| (node == 'none'):
            # Now find minimal nonzero value in this matrix
            minresist = np.min(possconnectmat[possconnectmat > 0])
            # Figure out where this value is
            locator = np.where(possconnectmat==minresist)
            # Redundant so I only spit out the first item. 
            return self.names[locator[0]]
            '''
            Note that this only returns the *first* pair (there could 
            be multiple pairs that have the same minimal resistance)
            If I wanted to check for more loops I'd just loop through
            the length of the locator.
            '''
        elif (node in self.names):
            #Figure out where in name list node is.
            nameloc = np.where(self.names == node)
            Col = possconnectmat[:,nameloc]
            minresist = np.min(Col[Col > 0])
            locator = np.where(Col==minresist)
            return self.names[locator[0]]
        else:
            raise ValueError("Input not valid name")

        
    def add_link(self, node1, node2):
        if (node1 in self.names) & (node2 in self.names):
            # Grab locators
            loc1 = np.where(self.names==node1)
            loc2 = np.where(self.names==node2)
            # Add link
            self.adj[loc1, loc2] = 1
            self.adj[loc2, loc1] = 1
        else:
            raise ValueError("Inputs are not valid names")

        
Soc = LinkPredictor("Data/social_network.csv")
print(Soc.predict_link('Alan'))
Soc.add_link('Sonia', 'Alan')
print(Soc.predict_link("Alan"))
print("HOORAY!")

['Sonia']
['Piers']
HOORAY!
